# 1 | Importıng Modules

In [34]:

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from IPython.display import clear_output

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import random

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

from keras import Sequential
from keras.layers import GlobalAveragePooling2D, Dropout, Dense, BatchNormalization, Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.callbacks import EarlyStopping, ModelCheckpoint


# 2 | EDA

In [ ]:
rootPath = 'dataset/Rice_Image_Dataset'
class_names = sorted(os.listdir(rootPath))[:-2]
class_names

In [ ]:
sizes = [len(os.listdir(rootPath + '/' + name)) for name in class_names]
sizes

In [ ]:
fig = px.pie(
     names= class_names,
     values = sizes,
     width = 450 ,
     title='Class Distribution',
     hole=0.4
)
fig.update_layout({'title':{'x':0.45}})
fig.show()


In [38]:
def load_random_img(dir,label):
    plt.figure(figsize=(10,10))
    i=0
    for label in class_names:
        i+=1
        plt.subplot(1,5,i)
        file = random.choice(os.listdir(f'{dir}/{label}'))
        print(file)
        image_path = os.path.join(f'{dir}/{label}', file)
        print(image_path)

        img=cv2.imread(image_path)
        plt.title(label)
        plt.imshow(img)
        plt.grid(None)
        plt.axis('off')


In [ ]:
for i in range(3):
    load_random_img(rootPath, class_names)


# 3 | Data Spliting

In [ ]:
pip install split-folders --break-system-packages

In [41]:
import splitfolders

#(Train_Val_Test)

splitfolders.ratio(rootPath, output='rice_imgs', seed=123, ratio=(.7,.15,.15), group_prefix=None)


In [42]:
batch_size = 128
img_height , img_width = 256,256
input_shape = (img_height, img_width,3)


In [43]:
# Data Pre-processing
# ImageDataGenerator can read images from disk and preprocess them into proper tensors.
# It will also set up generators that convert these images into batches of tensors—helpful when training the network.

datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
#After defining the generators for training and validation images, 
# the flow_from_directory method load images from the disk, applies rescaling,
# and resizes the images into the required dimensions.

Train = datagen.flow_from_directory(
            'rice_imgs/train',
            target_size = (img_height,img_width),
            batch_size = batch_size,
            subset = "training",
            class_mode='categorical')

Val = datagen.flow_from_directory(
            'rice_imgs/val',
            target_size = (img_height,img_width),
            batch_size = batch_size,
            class_mode='categorical',
            shuffle =False)

Test = datagen.flow_from_directory(
            'rice_imgs/test',
            target_size = (img_height, img_width),
            batch_size = batch_size,
            class_mode='categorical',
            shuffle=False)



# 4 | CNN MODEL

In [ ]:
CNN = tf.keras.models.Sequential()

# Convolution
CNN.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu',input_shape=[256,256,3]))

#Pooling
CNN.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

#Flattening
CNN.add(tf.keras.layers.Flatten())

#Full Conection
CNN.add(tf.keras.layers.Dense(units=512,activation='relu'))

#Output layer
CNN.add(tf.keras.layers.Dense(units=5, activation='sigmoid'))



In [46]:
CNN.compile(optimizer='adam', loss='binary_crossentropy' , metrics=['accuracy'])

In [ ]:
Model = CNN.fit(x=Train, validation_data=Val, epochs=3)

In [ ]:
CNN.summary()

In [ ]:
score = CNN.evaluate(Test, verbose=1)
print(f'Model Loss: {score[0]}, Accuracy: {score[1]*100}')

# 5 | Evaluation

In [21]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
Y_pred = CNN.predict(Test)
y_pred = np.argmax(Y_pred,axis=1)
print(Test)
confusion_mtx = confusion_matrix(y_pred,Test.classes)
f,ax = plt.subplots(figsize=(6, 6))
sns.heatmap(confusion_mtx, annot=True, 
            linewidths=0.01,
            linecolor="white", 
            fmt= '.1f',ax=ax, cmap=plt.cm.Accent_r)

plt.xlabel("Predicted Label")
plt.ylabel("True Label")
ax.xaxis.set_ticklabels(Test.class_indices)
ax.yaxis.set_ticklabels(class_names)
plt.title("Confusion Matrix")
plt.show()


In [ ]:
plt.figure(figsize=(10, 10))
#The next function returns a batch from the dataset. The return value is in form of (training features and labels)
x, label = next(Train)
for i in range(12):
    plt.subplot(3, 4, i+1)
    plt.imshow(x[i])
    result = np.where(label[i]==1)
    predict = CNN(tf.expand_dims(x[i], 0))
    score = tf.nn.softmax(predict[0])
    score_label = class_names[np.argmax(score)]
    plt.title(f'Truth: {class_names[result[0][0]]}\nPrediction:{score_label}')
    plt.axis(False)

In [ ]:
testImage = tf.keras.utils.load_img(rootPath+"/Basmati/basmati (21).jpg",target_size=(256,256))
testImage

In [ ]:
testImage = tf.keras.utils.img_to_array(testImage)
testImage = np.expand_dims(testImage,axis=0)
Result = CNN.predict(testImage)
print(Result)